In [1]:
import dash
from dash import html, dcc, Input, Output
import pandas as pd 
import plotly.express as px

In [2]:
df = pd.read_csv('FAOSTAT_data_en_nutrition.csv')
#df

In [3]:
df['Value'] = df['Value'].astype(str).str.replace("<", "", regex=False).astype(float)
df['Year'] = df['Year'].astype(str).str[:4].astype(int)
df_grouped = df.groupby(['Year','Area'], as_index=False)['Value'].mean()
df_grouped['Value'] = df_grouped['Value'].fillna(0)

In [4]:
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id = 'Area_category',
        options = [{"label": Area, "value": Area} for Area in df_grouped['Area'].unique()],
        multi = True,
        placeholder = "Select Area"
    ),
        dcc.Graph(id = 'line-chart')
])

@app.callback(
    Output('line-chart', 'figure'),
    Input('Area_category', 'value')
)

def update_chart(selected_area):
    if not selected_area:
         return px.line(title="Share of the population that is undernourished(Please select an area)")
    filtered_df = df_grouped[df_grouped['Area'].isin(selected_area)]
    y_min = filtered_df['Value'].min() * 0.95  
    y_max = filtered_df['Value'].max() * 1.05 
    fig = px.line(filtered_df, x='Year', y = 'Value', color='Area', title="Share of the population that is undernourished")
    fig.update_layout(yaxis=dict(range=[y_min, y_max]))
    return fig

if __name__ == '__main__' :
    app.run_server(debug=True)